In [1]:
from photozpy import HeaderCorrection, Combine, Telescope, Reduction, CollectionManager
from ccdproc import ImageFileCollection
from pathlib import Path
from astropy.time import Time
import numpy as np

from astropy.nddata import CCDData
from astropy.visualization import hist
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from astropy.stats import mad_std

from convenience_functions import show_image
from astropy.io import fits
#plt.style.use('guide.mplstyle')

from astropy.io import fits

from photozpy import CollectionManager

In [2]:
# define the telescope
telescope = Telescope(telescope = "RM", mode = "griz", ccd = "Andor")

In [3]:
# get the original 
path = Path("/Users/shengyong/conda_envs/photozpy/photozpy/tests/20210807_Examined_has_flats")
image_collection = ImageFileCollection(location = path, glob_include = "*.fits")

In [4]:
targets = {"B3_2238+410_": "B3 2238+410",
           "RX_J1533.3+3416": "RX J1533.3+3416", 
           "SA92-253": "SA92-253"}

In [5]:
header_edition = HeaderCorrection(image_collection = image_collection, telescope = telescope, target_dict = targets)

In [6]:
image_collection = header_edition.correct_headers_by_filename(save_location = "/Users/shengyong/conda_envs/photozpy/photozpy/tests/20210807")

Editing headers by file names......


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:04<00:00, 14.07it/s]


Header edition by file names completed!
----------------------------------------



In [7]:
image_collection = header_edition.correct_filter_headers()

Editting filter headers......


44it [00:01, 22.12it/s]


Filter header edition completed!
----------------------------------------



In [8]:
combine_images = Combine(image_collection, telescope)

In [9]:
combine_images.combine_bias_or_dark(image_type = "Bias")

Using ['Bias_009.fits', 'Bias_008.fits', 'Bias_003.fits', 'Bias_002.fits', 'Bias_006.fits', 'Bias_005.fits', 'Bias_001.fits', 'Bias_010.fits', 'Bias_007.fits', 'Bias_004.fits'] to combine the master Bias image!
Bias combined!
----------------------------------------



In [10]:
calibrate_images = Reduction(image_collection, telescope)

In [11]:
calibrate_images.apply_bias_correction()

Apply Master_Bias.fits correction to Flat_Sloan g'2_002.fits
Apply Master_Bias.fits correction to Flat_Sloan r'2_003.fits
Apply Master_Bias.fits correction to Flat_Sloan r'2_001.fits
Apply Master_Bias.fits correction to B3_2238+410_Sloan g'2_001.fits
Apply Master_Bias.fits correction to RX_J1533.3+3416_003.fits
Apply Master_Bias.fits correction to RX_J1533.3+3416_002.fits
Apply Master_Bias.fits correction to B3_2238+410_Sloan r'2_010.fits
Apply Master_Bias.fits correction to RX_J1533.3+3416_005.fits
Apply Master_Bias.fits correction to Flat_Sloan g'2_003.fits
Apply Master_Bias.fits correction to RX_J1533.3+3416_007.fits
Apply Master_Bias.fits correction to Dark_001.fits
Apply Master_Bias.fits correction to Dark_003.fits
Apply Master_Bias.fits correction to SA92-253_Sloan z'2_004.fits
Apply Master_Bias.fits correction to B3_2238+410_Sloan i'2_007.fits
Apply Master_Bias.fits correction to RX_J1533.3+3416_010.fits
Apply Master_Bias.fits correction to Flat_Sloan g'2_001.fits
Apply Master_B

In [12]:
combine_images.combine_bias_or_dark(image_type = "Dark")

Using ['Dark_001.fits', 'Dark_002.fits', 'Dark_003.fits'] to combine the master Dark image!
Dark combined!
----------------------------------------



In [13]:
calibrate_images.apply_dark_correction()

Apply Master_Dark.fits correction to Flat_Sloan g'2_002.fits
Apply Master_Dark.fits correction to Flat_Sloan r'2_003.fits
Apply Master_Dark.fits correction to Flat_Sloan r'2_001.fits
Apply Master_Dark.fits correction to B3_2238+410_Sloan g'2_001.fits
Apply Master_Dark.fits correction to RX_J1533.3+3416_003.fits
Apply Master_Dark.fits correction to RX_J1533.3+3416_002.fits
Apply Master_Dark.fits correction to B3_2238+410_Sloan r'2_010.fits
Apply Master_Dark.fits correction to RX_J1533.3+3416_005.fits
Apply Master_Dark.fits correction to Flat_Sloan g'2_003.fits
Apply Master_Dark.fits correction to RX_J1533.3+3416_007.fits
Apply Master_Dark.fits correction to SA92-253_Sloan z'2_004.fits
Apply Master_Dark.fits correction to B3_2238+410_Sloan i'2_007.fits
Apply Master_Dark.fits correction to RX_J1533.3+3416_010.fits
Apply Master_Dark.fits correction to Flat_Sloan g'2_001.fits
Apply Master_Dark.fits correction to SA92-253_Sloan g'2_001.fits
Apply Master_Dark.fits correction to B3_2238+410_Sl

In [14]:
combine_images.combine_light_or_flat(image_type = "Flat")

Using ["Flat_Sloan g'2_002.fits", "Flat_Sloan g'2_001.fits", "Flat_Sloan g'2_003.fits"] to combine the master Flat image in SDSS_g' filter!
Flat in SDSS_g' filter combined!
----------------------------------------------------------

Using ["Flat_Sloan r'2_002.fits", "Flat_Sloan r'2_001.fits", "Flat_Sloan r'2_003.fits"] to combine the master Flat image in SDSS_r' filter!
Flat in SDSS_r' filter combined!
----------------------------------------------------------

Using ["Flat_Sloan i'2_003.fits", "Flat_Sloan i'2_002.fits", "Flat_Sloan i'2_001.fits"] to combine the master Flat image in SDSS_i' filter!
Flat in SDSS_i' filter combined!
----------------------------------------------------------

Using ["Flat_Sloan z'2_002.fits", "Flat_Sloan z'2_003.fits", "Flat_Sloan z'2_001.fits"] to combine the master Flat image in SDSS_z' filter!
Flat in SDSS_z' filter combined!
----------------------------------------------------------



In [15]:
calibrate_images.apply_flat_correction()

Using Master_Flat_SDSS_g'.fits correcting B3_2238+410_Sloan g'2_009.fits
Using Master_Flat_SDSS_g'.fits correcting RX_J1533.3+3416_005.fits
Using Master_Flat_SDSS_g'.fits correcting RX_J1533.3+3416_009.fits
Using Master_Flat_SDSS_g'.fits correcting B3_2238+410_Sloan g'2_005.fits
Using Master_Flat_SDSS_g'.fits correcting B3_2238+410_Sloan g'2_001.fits
Using Master_Flat_SDSS_g'.fits correcting RX_J1533.3+3416_001.fits
Using Master_Flat_SDSS_g'.fits correcting SA92-253_Sloan g'2_005.fits
Using Master_Flat_SDSS_g'.fits correcting SA92-253_Sloan g'2_001.fits
Using Master_Flat_SDSS_r'.fits correcting B3_2238+410_Sloan r'2_010.fits
Using Master_Flat_SDSS_r'.fits correcting B3_2238+410_Sloan r'2_006.fits
Using Master_Flat_SDSS_r'.fits correcting SA92-253_Sloan r'2_006.fits
Using Master_Flat_SDSS_r'.fits correcting SA92-253_Sloan r'2_002.fits
Using Master_Flat_SDSS_r'.fits correcting B3_2238+410_Sloan r'2_002.fits
Using Master_Flat_SDSS_r'.fits correcting RX_J1533.3+3416_010.fits
Using Master_F